# Short Keras Recap

The Keras network developed in this notebook serves as the basis for our subsequent experiments. It is intentionally kept short and concise and is intended as a reference point (and chance to experiment) for those who see TensorFlow code for the first time. In the next step we will train the Keras network developed in this notebook with a DP Optimmizer from the Tensorflow-Privacy Library.

## Imports (and Google Drive Mount in Google Colab)

!!! For more fun in Goole Colab don't forget to select Runtime > Change runtime type > GPU

In [1]:
# Set tensorlfow version in google colab
try:
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow.compat.v1 as tf

# Update tensorflow-privacy
!pip install tensorflow-privacy --upgrade

import numpy as np

# Mount google drive to save trained models, experiment results and cat pics
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

TensorFlow 1.x selected.
     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 296kB 10.7MB/s 
  Found existing installation: tensorflow-privacy 0.2.2
    Uninstalling tensorflow-privacy-0.2.2:
      Successfully uninstalled tensorflow-privacy-0.2.2
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Function Definitions

### Load and Preprocess MNIST Dataset

In [ ]:
def load_mnist():
  """Loads and preprocesses the MNIST dataset.

  Returns
  -------
  tuple
      (training data, training labels, test data, test labels)
  """
  train, test = tf.keras.datasets.mnist.load_data()
  train_data, train_labels = train
  test_data, test_labels = test

  # Normalize the data to a range between 0 and 1
  train_data = np.array(train_data, dtype=np.float32) / 255
  test_data = np.array(test_data, dtype=np.float32) / 255

  # Reshape the images to (28, 28, 1)
  train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
  test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)

  # Convert a class vector (integers) to binary class matrix (one-hot enc.)
  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

  return train_data, train_labels, test_data, test_labels

### Create a simple CNN Model

In [ ]:
def create_model():
  """Creates a simple example CNN.

  Returns
  -------
  tensorflow.python.keras.engine.sequential.Sequential
      A simple example CNN
  """
  from tensorflow.keras import Sequential
  from tensorflow.keras.layers import Conv2D
  from tensorflow.keras.layers import MaxPool2D
  from tensorflow.keras.layers import Flatten
  from tensorflow.keras.layers import Dense

  model = Sequential()
  model.add(Conv2D(filters=16, kernel_size=8, strides=2, padding='same',
                   activation='relu', input_shape=(28, 28, 1)))
  model.add(MaxPool2D(2, 1))
  model.add(Conv2D(32, 4, strides=2, padding='valid', activation='relu'))
  model.add(MaxPool2D(2, 1))
  model.add(Flatten())
  model.add(Dense(32, activation='relu'))
  model.add(Dense(10))

  return model

## Fit a Model

In [ ]:
train_data, train_labels, test_data, test_labels = load_mnist()

model = create_model()

# Hyper parameter
learning_rate = 0.05
batch_size = 64
epochs = 20

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    validation_data=(test_data, test_labels),
                    batch_size=batch_size,
                    verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 10s 170us/sample - loss: 0.3602 - acc: 0.8872 - val_loss: 0.0877 - val_acc: 0.9711
Epoch 2/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0873 - acc: 0.9738 - val_loss: 0.0601 - val_acc: 0.9813
Epoch 3/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0625 - acc: 0.9805 - val_loss: 0.0412 - val_acc: 0.9874
Epoch 4/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0515 - acc: 0.9844 - val_loss: 0.0436 - val_acc: 0.9858
Epoch 5/20
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0432 - acc: 0.9863 - val_loss: 0.0381 - val_acc: 0.9878
Epoch 6/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0382 - acc: 0.9881 - val_loss: 0.0336 - val_acc: 0.9891
Epoch 7/20
60000/60000 [==============================] - 4s 74us/sample - loss: 0.0331 - acc: 0.9897 - val_loss: 

In [ ]:
# It is that easy to save the whole model
model.save('drive/My Drive/glorious_digit_model.h5')

# and also loading is fun
from tensorflow.keras.models import load_model
model = load_model('drive/My Drive/glorious_digit_model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
